# Analisis Data Zuber

Anda bekerja sebagai seorang analis untuk Zuber, sebuah perusahaan berbagi tumpangan (ride-sharing) baru yang diluncurkan di Chicago. Tugas Anda adalah untuk menemukan pola pada informasi yang tersedia. Anda ingin memahami preferensi penumpang dan dampak faktor eksternal terhadap perjalanan. Dengan menggunakan basis data, Anda akan menganalisis data dari kompetitor dan menguji hipotesis terkait pengaruh cuaca terhadap frekuensi perjalanan.

## Tujuan

Proyek ini berfokus pada analisis data perusahan taksi. Data sebelumnya telah dilakukan pre-processing untuk membersihkan dan merapihkan data agar dapat diolah. Data yang diperoleh diolah dengan SQL. Untuk analisis ini, selanjutnya dilakukan Exploratory Data Analysis (EDA) dan Data Visualization untuk mengetahui insight dari data. Dan terakhir dilakukan analisis data statistik untuk Hypothesis Testing.

### Hipotesis

1. Durasi rata-rata perjalanan dari Loop ke Bandara Internasional O'Hare berubah pada hari-hari Sabtu yang hujan.

### Tahapan

Data tentang hasil SQL disimpan dalam file:
* `/datasets/project_sql_result_01.csv`
* `/datasets/project_sql_result_04.csv`
* `/datasets/project_sql_result_07.csv`

Proyek ini terdiri dari dua tahap:

1. Analisis Data Exploratif (EDA)
2. Analisis Data Statistik (Hyphothesis Testing)


## 1. Analisis Data Exploratif (EDA)

**Import *library* yang dibutuhkan:**

In [ ]:
# Import library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st

### 1.1. Memuat Data

In [ ]:
# Try and except datasets
try:
    company_data = pd.read_csv('moved_project_sql_result_01.csv')
    dropoff_data = pd.read_csv('moved_project_sql_result_04.csv')
    travel_data = pd.read_csv('moved_project_sql_result_07.csv')
except:
    company_data = pd.read_csv('/datasets/project_sql_result_01.csv')
    dropoff_data = pd.read_csv('/datasets/project_sql_result_01.csv')
    travel_data = pd.read_csv('/datasets/project_sql_result_01.csv')

### 1.2. Mengeksplorasi Data Awal

#### **Data Perusahaan**

In [ ]:
company_data

**Deskripsi company_data**: 


- `company_name` — nama perusahaan taksi
- `trips_amount` —  jumlah perjalanan untuk setiap perusahaan taksi pada tanggal 15-16 November 2017

In [ ]:
# Check data info
company_data.info()

In [ ]:
# Check statistics data
company_data.describe()

#### **Data Dropoff Location**

In [ ]:
dropoff_data

**Deskripsi dropoff_data**: 


- `dropoff_location_name` — nama wilayah di Chicago tempat perjalanan berakhir
- `average_trips` —  jumlah rata-rata perjalanan yang berakhir di setiap wilayah pada bulan November 2017

In [ ]:
# Check data info
dropoff_data.info()

In [ ]:
# Check statistics data
dropoff_data.describe()

In [ ]:
# Check duplicated data
dropoff_data.duplicated().sum()

### 1.3. Mempelajari Parameter Inti 

#### Identifikasi 10 wilayah teratas

In [ ]:
top_ten = dropoff_data.sort_values(by='average_trips', ascending=False).head(10)
top_ten

In [ ]:
# Create a bar plot
plt.figure(figsize=(8,6))
plt.xticks(rotation=45)
sns.barplot(x="dropoff_location_name", y="average_trips", data=top_ten, color='#BCD8C1')
plt.title('Top Ten Dropoff Location')
plt.show()

**Kesimpulan sementara:**

* Sepuluh dropoff location yang memiliki rata-rata perjalanan tertinggi yaitu di lokasi Loop, River North, Streeterville, West Loop, O'Hare, Lake View, Grant Park, Museum Campus, Gold Coast, dan Sheffield & DePaul.

#### Grafik Perusahaan Taksi

In [ ]:
company_data.head(20)

* Untuk mempermudah analisa, perusahaan tertinggi yang akan diambil yaitu perusahaan dengan perjalanan yang diatas 5000.

In [ ]:
top_company = company_data.sort_values(by='trips_amount', ascending=False).head(11)

In [ ]:
# Create a bar plot
plt.figure(figsize=(8,6))
sns.barplot(x="trips_amount", y="company_name", data=top_company, color='#BCD8C1')
plt.title('Top Cabs Company')
plt.show()

**Kesimpulan:**

* Perusahaan tertinggi yaitu Flash Cab dengan jumlah perjalanan 19558 perjalanan. Kemudian Taxi Affiliation Services dengan 11422 perjalanan. Kemudian Medallion Leasin dengan 10367 perjalanan dan Yellow Cab dengan 9888 perjalanan.

## 2. Analisis Data Statistik (Hypothesis Testing)

### 2.1. Mengeksplorasi Data Perjalanan dari Loop ke Bandara Internasional O'Hare

**Hipotesis:**

**Durasi rata-rata perjalanan dari Loop ke Bandara Internasional O'Hare berubah pada hari-hari Sabtu yang hujan.**

Sebelum menguji perlu ditemukan data-data yang mendukung hipotesis tersebut.

In [ ]:
travel_data

**Deskripsi travel_data**: 

- `start_ts` — tanggal dan waktu penjemputan
- `weather_conditions` —  kondisi cuaca saat perjalanan dimulai
- `duration_seconds` —  durasi perjalanan dalam satuan detik

In [ ]:
travel_data.info()

data pada kolom start_ts diubah menjadi datetime dan menambahkan beberapa kolom baru

In [ ]:
# Cast 'start_ts' column to datetime
travel_data['start_ts'] = pd.to_datetime(travel_data['start_ts'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
travel_data['start_date'] = travel_data['start_ts'].dt.date
travel_data['start_time'] = travel_data['start_ts'].dt.time
travel_data['start_day'] = travel_data['start_ts'].dt.day_name()

In [ ]:
travel_data.info()

In [ ]:
#creates slice for the weather condition that "Good" 
good_weather = travel_data.query('weather_conditions == "Good"')

In [ ]:
#creates slice for the weather condition that "Bad" 
bad_weather = travel_data.query('weather_conditions == "Bad"')

In [ ]:
travel_data

### 2.2. Menguji Hipotesis

**Hipotesis:**

**Durasi rata-rata perjalanan dari Loop ke Bandara Internasional O'Hare berubah pada hari-hari Sabtu yang hujan.**

Dalam uji variansi, jika rasio sampel dengan varians yang lebih besar dibandingkan dengan sample yang variansnya lebih kecil <4, maka bisa kita assumsikan varians keduanya setara. Atau bisa menggunakan uji levene dimana H0 nya adalah kedua grup memiliki varians setara dan H1 nya adalah kedua grup memiliki varians yang berbeda. Jika di hasil levene test menunjukkan p-value < dari critical value (alpha) maka kita bisa menolak H0, dan sebaliknya.

In [ ]:
# Create a sample for levene test
sample_1 = good_weather['duration_seconds']
sample_2 = bad_weather['duration_seconds']

In [ ]:
# Levene test
st.levene(sample_1, sample_2)

Dikarenakan p-value > dari critical value (alpha = 0.05) maka kita tidak bisa menolak H0 dan kedua sampel dianggap memiliki varians yang sama (equal_var = True)

* H0 (Null Hypothesis) : Durasi rata-rata perjalanan dari Loop ke Bandara Internasional O'Hare sama pada hari-hari Sabtu yang hujan.
* H1 (Alternative Hypothesis) : Durasi rata-rata perjalanan dari Loop ke Bandara Internasional O'Hare berubah pada hari-hari Sabtu yang hujan.

In [ ]:
# ttest_ind for sample 1 and 2
alpha = 0.05

results = st.ttest_ind(sample_1, sample_2, equal_var=True)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("Kita menolak hipotesis nol")
else:
    print("Kita tidak dapat menolak hipotesis nol")

**Kesimpulan:**

* Hasil yang didapat dari uji hipotesis yaitu Durasi rata-rata perjalanan dari Loop ke Bandara Internasional O'Hare berubah pada hari-hari Sabtu yang hujan adalah **BENAR**.

## Kesimpulan Umum

#### A. Analisis Data Esksploratif (EDA)

Dari hasil analisis data esksploratif, dapat disimpulkan bahwa:
1. Data sudah dilakukan pra-pemrosesan data dengan SQL, sehingga data yang diperoleh sudah bersih dan rapih.
2. Wilayah dengan rata-rata perjalanan yang paling tinggi yaitu di lokasi Loop, River North, Streeterville, West Loop, O'Hare, Lake View, Grant Park, Museum Campus, Gold Coast, dan Sheffield & DePaul.
3. Perusahaan taksi yang paling banyak melakukan perjalanan yaitu Flash Cab. Kemudian Taxi Affiliation Services dan Medallion Leasin.

#### B. Analisis Data Statistik (Hypothesis Testing)

Dari hasil analisis pengujian hipotesis, dapat disimpulkan bahwa:
1. Hasil yang didapat dari uji hipotesis yaitu Durasi rata-rata perjalanan dari Loop ke Bandara Internasional O'Hare berubah pada hari-hari Sabtu yang hujan adalah benar.